### ADK Agent Engine 배포

In [2]:
import os
from dotenv import load_dotenv
import argparse

import vertexai
from vertexai import agent_engines
from vertexai.agent_engines import AdkApp

from agent import root_agent

In [3]:
# 환경 변수 로드.
load_dotenv()

from dotenv import dotenv_values
config = dotenv_values("../.env")
for key, value in config.items():
    print(f"{key}: {value}")

GOOGLE_GENAI_USE_VERTEXAI: TRUE
GOOGLE_CLOUD_PROJECT: ai-hangsik
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_GENAI_MODEL: gemini-2.5-flash
GOOGLE_GENAI_LIVE_MODEL: gemini-live-2.5-flash
AGENT_ENGINE_BUCKET: gs://agent_engine_0120


In [ ]:
# Vertex AI 환경 초기화 : Agent Engine을 배포하기 위한 환경 설정
# Agent Engine 과 Gemini 모델을 사용하기 위한 리전을 다르게 할 수 있음.
vertexai.init(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket=os.getenv("AGENT_ENGINE_BUCKET"),
)

In [5]:
agent_name = "engine_0122"

In [6]:
# adk_app 생성
adk_app = AdkApp(agent=root_agent)

# Agent Engine 환경 설정
display_name = agent_name
gcs_dir_name = os.getenv("AGENT_ENGINE_BUCKET")
description = "사용자 질문에 대한 AI 정보 검색 어시스턴트"
requirements = [
    "google-adk[vertexai]",
    "google-cloud-aiplatform[adk,agent-engines]",
    "cloudpickle==3.0",
    "python-dotenv",
]

In [7]:

env_vars = {
  "GOOGLE_CLOUD_AGENT_ENGINE_ENABLE_TELEMETRY": "true",
  "OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT": "true",
}

create = False

if create:
    # 최초 Agent Engine을 디플로이.
    remote_agent = agent_engines.create(
                    adk_app,
                    display_name=display_name,
                    gcs_dir_name = gcs_dir_name,
                    description=description,
                    requirements=requirements,
                    extra_packages = []
    )
else:
    # 이미 디플로이된 Agent를 업데이트.
    # 아래 Resource name은 Console 에서 확인 가능.
    agent_engine_resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/6119737684177977344"
    
    remote_agent = agent_engines.update(
        resource_name = agent_engine_resource_name,
        agent_engine = adk_app
    )    

Using bucket agent_engine_0120
Wrote to gs://agent_engine_0120/agent_engine/agent_engine.pkl
Update Agent Engine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/6119737684177977344/operations/2251425137918214144
Agent Engine updated. Resource name: projects/721521243942/locations/us-central1/reasoningEngines/6119737684177977344


### Run Agent

In [11]:

agent_engine_resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/6119737684177977344"

remote_agent = agent_engines.get(agent_engine_resource_name)
print(f"Display name : {remote_agent.display_name}")
print(f"Reource name : {remote_agent.resource_name}")


remote_session = await remote_agent.async_create_session(user_id="u_456")
print(remote_session)



query = "생성형 AI란 무엇인가요?"

print(f"\n 👤 User: {query}\n")

# async for event in remote_agent.async_stream_query(
#     user_id="u_456",
#     session_id=remote_session["id"],
#     message="whats the weather in new york",
# ):
#     print(event)

events = remote_agent.stream_query(user_id="u_456", 
                                   session_id=remote_session["id"],
                                   message=query)

print(events)                                   
for event in events:
    print(event)
    response = event['content']['parts'][0]['text']
    print(f"\n 🤖 Local AI Assistant: {response}\n")


Display name : engine_0121
Reource name : projects/721521243942/locations/us-central1/reasoningEngines/6119737684177977344
{'appName': '6119737684177977344', 'events': [], 'state': {}, 'id': '1702677043616940032', 'userId': 'u_456', 'lastUpdateTime': 1768956403.735912}

 👤 User: 생성형 AI란 무엇인가요?

<generator object _wrap_stream_query_operation.<locals>._method at 0x129da98b0>
{'model_version': 'gemini-2.5-flash', 'content': {'parts': [{'text': '생성형 AI(Generative AI 또는 GenAI)는 인공지능(AI)의 한 분야로, 기존 데이터를 학습하여 텍스트, 이미지, 오디오, 비디오, 소프트웨어 코드 등 새롭고 독창적인 콘텐츠를 생성하는 기술을 의미합니다.\n\n주요 특징은 다음과 같습니다.\n*   **콘텐츠 생성 능력** 기존 데이터를 단순히 분석하거나 예측하는 것을 넘어, 학습한 패턴과 구조를 기반으로 완전히 새로운 결과물을 만들어냅니다.\n*   **작동 방식** 주로 딥러닝과 신경망 기술을 활용하며, 대규모 데이터셋을 통해 학습하여 데이터 내의 잠재적인 패턴과 관계를 파악합니다. 사용자는 주로 자연어 프롬프트(지시문)를 통해 원하는 결과물을 AI에게 요청합니다.\n*   **다양한 활용 분야** 텍스트 생성(대화, 스토리, 코드), 이미지 생성(예술 작품, 사실적인 사진), 오디오 및 비디오 생성 등 여러 형태의 콘텐츠를 만들 수 있습니다.\n*   **주요 모델** 생성형 AI에 사용되는 대표적인 모델로는 생성적 적대 신경망(GAN), 변이형 오토인코더(VAE), 트랜스포머(Transformer), 확산

In [ ]:
import requests
import google.auth
import google.auth.transport.requests

credentials, project = google.auth.default()
auth_request = google.auth.transport.requests.Request()
credentials.refresh(auth_request)
token = credentials.token

region = os.getenv("GOOGLE_CLOUD_LOCATION")

resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/6119737684177977344"
url = f"https://{region}-aiplatform.googleapis.com/v1/{resource_name}:streamQuery?alt=sse"

payload = {
    "class_method": "async_stream_query",
    "input": {
      "user_id": "u_456",
      "message": "What is the exchange rate from US dollars to KRW today?",
    }
  }

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# 4. 호출 및 응답 확인
response = requests.post(url, json=payload, headers=headers)
print(response.json())

{'model_version': 'gemini-2.5-flash', 'content': {'parts': [{'text': '오늘 미국 달러(USD) 대비 대한민국 원화(KRW)의 환율은 1 USD당 약 ₩1,478.03입니다. 이 환율은 02:47 UTC 기준 중간 시장 환율이며, 글로벌 통화 시장에서 매수 및 매도 가격의 중간 지점을 나타냅니다.\n\n환율은 실시간으로 변동될 수 있습니다. 최근에는 원화가 달러 대비 1,400원 수준으로 강세를 보일 것이라는 전망도 있었습니다.'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'xe.com', 'title': 'xe.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEoYBtO2ebTz30YvjMfnMK6FExR3ntWCnoCJdT6m8hgVN8MgxYGOf6kmAVVvw10pmwA5LFgbINjyMAKJ6VdvqGEkeZPsac71Ipp0vo75TWgDrvYdypNsOaJTaboOnDX_jvOlb76tlBW58m9fEGLqmlgT6_bsO7wnwysZdqXY6mUvu8='}}, {'web': {'domain': 'westernunion.com', 'title': 'westernunion.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHUULiVPbcBNpRCpXUKsxSyN573Ocz8rAfpax02_gtizNgVwOn5dfT6LaeOk3wmN6rX8ZxHzXGsJESzndbGvlAT9Os1x9UNOxk-GYz0U_j-VcOkZs_2WW1cLvP_hku45SGYK3UW2edi09v-K3eZqpOH47ddYhb_O0zZTE0-BxdtMHu1fi-t'}}, {'web': {'domain': 'xtransfer.

In [ ]:
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://LOCATION-aiplatform.googleapis.com/v1/projects/PROJECT_ID/locations/LOCATION/reasoningEngines/RESOURCE_ID:streamQuery?alt=sse -d '{
  "class_method": "async_stream_query",
  "input": {
    "user_id": "USER_ID",
    "session_id": "SESSION_ID",
    "message": "What is the exchange rate from US dollars to SEK today?",
  }
}'